In [1]:
# Imports

import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
from glob import glob
from matplotlib import pyplot as plt
from datetime import datetime

from __future__ import print_function
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import model_from_json
import os

/Users/yhcu/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Directory Path:
IMG_PATH = "../data/pets/train/"
LABEL_PATH = "../data/pets/train_label.txt"
TRAIN_VAL_SPLIT = 0.8

# Constants
NUM_IMG = 2000

In [3]:
# Image Standardization
def get_resized_images(path):
    imlist = []
    for i in range(NUM_IMG):
        path_str = path + "pet" + str(i+1) + ".jpg"
        im = cv2.imread(path_str)
        resized_im = cv2.resize(im, (64, 64))
        imlist.append(resized_im)  
    return imlist

# Read In Labels
def get_labels(path):
    labels = np.loadtxt(path, dtype = 'str')
    np.reshape(labels, (-1, 1))
    labels = np.asarray(labels)
    tmp = pd.get_dummies(labels)
    int_labels = tmp.values.argmax(1)
    return int_labels
    

In [4]:
start_time = datetime.now()
print("Reading Images @ " + str(start_time))
imlist = get_resized_images(IMG_PATH) # imlist contains a list of image numpy arrays
end_time = datetime.now()
print("Complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Reading Images @ 2018-03-02 19:05:43.727236
Complete @ 2018-03-02 19:05:51.600681 Time Cost: 0:00:07.873445


In [5]:
print(imlist[0].shape)

(64, 64, 3)


In [6]:
labels = get_labels(LABEL_PATH)
print(labels.shape)
print(labels[0:20])

(2000,)
[1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1]


In [7]:
X = np.asarray(imlist)
y = labels

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (1600, 64, 64, 3)
1600 train samples
400 test samples


In [9]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=[64, 64, 3]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [12]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

datagen.fit(X_train)

model.fit_generator(datagen.flow(X_train, y_train,
                    batch_size=128),
                    epochs=200,
                    validation_data=(X_test, y_test),
                    workers=4)

Epoch 1/200
13/13 [==============================]13/13 [==============================] - 62s 5s/step - loss: 0.7357 - acc: 0.6555 - val_loss: 0.6461 - val_acc: 0.7000

Epoch 2/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.6392 - acc: 0.6682 - val_loss: 0.6273 - val_acc: 0.7000

Epoch 3/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.6289 - acc: 0.6699 - val_loss: 0.6067 - val_acc: 0.7000

Epoch 4/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.6201 - acc: 0.6688 - val_loss: 0.5918 - val_acc: 0.7000

Epoch 5/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.6124 - acc: 0.6649 - val_loss: 0.6206 - val_acc: 0.7000

Epoch 6/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.6161 - acc: 0.6671 - val_loss: 0.5784


Epoch 49/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.4561 - acc: 0.7795 - val_loss: 0.5016 - val_acc: 0.7625

Epoch 50/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.4862 - acc: 0.7674 - val_loss: 0.5141 - val_acc: 0.7625

Epoch 51/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.4768 - acc: 0.7820 - val_loss: 0.4715 - val_acc: 0.7625

Epoch 52/200
13/13 [==============================]13/13 [==============================] - 61s 5s/step - loss: 0.4566 - acc: 0.7838 - val_loss: 0.4758 - val_acc: 0.7700

Epoch 53/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.4388 - acc: 0.8003 - val_loss: 0.4427 - val_acc: 0.8000

Epoch 54/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.4396 - acc: 0.8040 - val_loss:


Epoch 97/200
13/13 [==============================]13/13 [==============================] - 59s 5s/step - loss: 0.3468 - acc: 0.8508 - val_loss: 0.5064 - val_acc: 0.7975

Epoch 98/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.3501 - acc: 0.8439 - val_loss: 0.4999 - val_acc: 0.8000

Epoch 99/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.3370 - acc: 0.8497 - val_loss: 0.4785 - val_acc: 0.8100

Epoch 100/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.3220 - acc: 0.8576 - val_loss: 0.5346 - val_acc: 0.8100

Epoch 101/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.3316 - acc: 0.8511 - val_loss: 0.4771 - val_acc: 0.8250

Epoch 102/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.3567 - acc: 0.8397 - val_lo

13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.1881 - acc: 0.9267 - val_loss: 0.5841 - val_acc: 0.8125

Epoch 192/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.1803 - acc: 0.9249 - val_loss: 0.5816 - val_acc: 0.8350

Epoch 193/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.1771 - acc: 0.9303 - val_loss: 0.5759 - val_acc: 0.8200

Epoch 194/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.1622 - acc: 0.9271 - val_loss: 0.6080 - val_acc: 0.8100

Epoch 195/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.2045 - acc: 0.9103 - val_loss: 0.6409 - val_acc: 0.8350

Epoch 196/200
13/13 [==============================]13/13 [==============================] - 60s 5s/step - loss: 0.1934 - acc: 0.9183 - val_loss: 0.5868 -

In [13]:
model_json = model.to_json()
with open("../output/additionals/cnn_128_128.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../output/additionals/cnn_128_128.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
# Test model reading


model_json_file = open('../output/additionals/cnn_128_128.json', 'r')
loaded_model_json = model_json_file.read()
model_json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('../output/additionals/cnn_128_128.h5')
print("Loaded model from disk")
 
print("Evaluating test data @ " + str(datetime.now()))
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print(score)
print("Test data evaluation complete @ " + str(datetime.now()))

Loaded model from disk
Evaluating test data @ 2018-03-02 23:14:46.391908
[0.6347499346733093, 0.82]
Test data evaluation complete @ 2018-03-02 23:14:51.085406
